**1-Entrenamiento**

1.1 Preprocesado ejemplos

1.2 Construcción modelo a priori de la etiqueta (cada nivel de calidad del aire tiene un número difernte de ejemplos)

1.3 Construir distintos modelos de verosimilitud

Naive Bayes 

MVN

GMM

1.4 Construir sistemas con preprocesado, modelo a priori y cada uno de los modelos de verosimilitud

Sistema con Naive Bayes 

Sistema con MVN

Sistema con GMM

1.5 Comparar los sistema con un conjunto de validación  (elegir uno de los sistemas)

1.6 Almacenamiento objetos con Pickle

**2-Implementación e inferencia**

Creación script inferencia

2.1 Cargar objetos guardados del sistema seleccionado

2.2 Cargar conjunto de datos de test

2.3 Ejecución objetos guardados y guardado etiquetas predichas en un CSV con el mismo formato que practica_Y_train.csv

**3-Clustering**

3.1 Realizacion de clastering sobre el conjunto de datos practica_X_train.csv

3.2.1 ¿Cuantos clusters has elegido?¿Elección relacionada con el número de clases? 

3.2.2 Comparacion etiqueta de cada ejemplo de entrenamiento con el cluster asignado ¿Es posible crear una relación entre ambas?

3.2.3 ¿A que cluster se le asignaría cada ejemplo del conjunto de test si se crea una relacion entre cluster y etiqueta? ¿Qué etiqueta asignarias a cada ejemplo de test?

3.2.4 Si tenemos etiquetas asociadas a los datos ¿Se puede utilizar clustering para hacer clasificación?